# torch_scatter
+ scatter
+ segment_coo
+ segment_csr  

文档:[https://pytorch-scatter.readthedocs.io/en/latest/functions/scatter.html](https://pytorch-scatter.readthedocs.io/en/latest/functions/scatter.html)

## scatter

![](https://raw.githubusercontent.com/rusty1s/pytorch_scatter/master/docs/source/_figures/add.svg?sanitize=true)

将原始输入按照index向量分组处理，函数会自适应输入的维度，在进行分组的维度上，缩小为分组数大小的尺寸   
方法选项{"sum"(default),"mul","mean","min","max"}

In [11]:
import torch
from torch_scatter import scatter

src = torch.ones(10, 6)
index_0 = torch.tensor([0, 1, 0, 1, 2, 1, 3, 3, 3, 3])
index_1 = torch.tensor([0, 1, 0, 1, 2, 1])

print("第0维上分4组")
out_0 = scatter(src, index_0, dim=0, reduce="sum")
print(out_0.shape)
print(out_0)

print("第1维上分3组")
out_1 = scatter(src, index_1, dim=1, reduce="sum")
print(out_1.shape)
print(out_1)

第0维上分4组
torch.Size([4, 6])
tensor([[2., 2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3., 3.],
        [1., 1., 1., 1., 1., 1.],
        [4., 4., 4., 4., 4., 4.]])
第1维上分3组
torch.Size([10, 3])
tensor([[2., 3., 1.],
        [2., 3., 1.],
        [2., 3., 1.],
        [2., 3., 1.],
        [2., 3., 1.],
        [2., 3., 1.],
        [2., 3., 1.],
        [2., 3., 1.],
        [2., 3., 1.],
        [2., 3., 1.]])


p.s. 分组个数根据索引最大值确定，如果中间缺值，则会视为该组为空，返回0

In [13]:
src = torch.ones(10, 6)
index_1 = torch.tensor([0, 2, 0, 3, 0, 3])
out_1 = scatter(src, index_1, dim=1, reduce="sum")
print(out_1.shape)
print(out_1)

torch.Size([10, 4])
tensor([[3., 0., 1., 2.],
        [3., 0., 1., 2.],
        [3., 0., 1., 2.],
        [3., 0., 1., 2.],
        [3., 0., 1., 2.],
        [3., 0., 1., 2.],
        [3., 0., 1., 2.],
        [3., 0., 1., 2.],
        [3., 0., 1., 2.],
        [3., 0., 1., 2.]])


## COO 和 CSR 是稀疏矩阵存取格式

## segment_coo

src维度：$(x_0,...,x_{m-1},x_m,...,x_n)$  
index维度：$(x_0,...,x_{m-1},x_m)$  
out维度：$(x_0,...,x_{m-1},y,x_m,...,x_n)$  
对index将分组维度前的维度，使用view置1，要分组的维度置-1

index的值必须是递增的，不递增的情况下结果出错，或程序崩溃。该函数按照递增关系求分组，连续几个相同的索引值会被分到一组，如下例中的处理过程为  
[0]  
[0] [1 1 1 1]  
[0] [1 1 1 1] [2]  

使用segment_coo一般比scatter快

In [4]:
import torch
from torch_scatter import segment_coo

src = torch.ones(10, 6)
index = torch.tensor([0, 1, 1, 1, 1, 2])
index = index.view(1, -1)  # Broadcasting in the first and last dim.

out = segment_coo(src, index, reduce="sum")

print(out.size())
print(out)

torch.Size([10, 3])
tensor([[1., 4., 1.],
        [1., 4., 1.],
        [1., 4., 1.],
        [1., 4., 1.],
        [1., 4., 1.],
        [1., 4., 1.],
        [1., 4., 1.],
        [1., 4., 1.],
        [1., 4., 1.],
        [1., 4., 1.]])


## segment_csr

src维度：$(x_0,...,x_{m-1},x_m,...,x_n)$  
index维度：$(x_0,...,x_{m-1},y)$  
out维度：$(x_0,...,x_{m-1},y-1,x_m,...,x_n)$

对index将分组维度前的维度，使用view置1，分组维度置-1
该函数将索引值视为分组起点，在下例中为  
[0] [1 -] [3 -] [5] [6]  

segment_csr相比来说是最快的

In [6]:
from torch_scatter import segment_csr

src = torch.ones(10, 6)
indptr = torch.tensor([0, 1, 3, 5, 6])
indptr = indptr.view(1, -1)  # Broadcasting in the first and last dim.

out = segment_csr(src, indptr, reduce="sum")

print(out.size())
print(out)

torch.Size([10, 4])
tensor([[1., 2., 2., 1.],
        [1., 2., 2., 1.],
        [1., 2., 2., 1.],
        [1., 2., 2., 1.],
        [1., 2., 2., 1.],
        [1., 2., 2., 1.],
        [1., 2., 2., 1.],
        [1., 2., 2., 1.],
        [1., 2., 2., 1.],
        [1., 2., 2., 1.]])
